# 19 - Bitwise Operators

---

The previous chapter discussed dealing with binary files. When binary files are used, you are no longer working with characters and numbers; rather, you are working with bytes. To manipulate information on the level of bytes, Python offers a number of so-called "bitwise operators." You will not need these often, but when you delve into binary file manipulation, they might come in handy.

---

## Bits and bytes

A bit is the smallest size data unit that a computer can handle. A single bit can have only two different values, namely 1 and zero.

While "prehistoric" computers were indeed programmed by directly dealing with single ones and zeroes, very quickly computers were introduced that handled groups of bits. The smallest unit in that respect is the "byte", which consists of 8 bits. Today, the concept of a byte still permeates most computer languages, even though computers have been enhanced to use larger collections of bytes as smallest data units (notably, most computers today either deal with 32-bits or 64-bits data units).

### Binary counting

A byte consists of 8 bits, which you can display as a sequence of ones and zeroes, e.g., `11010010`. As such, a byte can be used to represent a number in binary code. If a byte is used to represent a positive number, that number can be calculated by multiplying the right-most bit by 1, the bit next to that by 2, the bit next to that by 4, etcetera, and adding up all those values. For instance, the sequence `11010010` is `1*128+1*64+0*32+1*16+0*8+0*4+1*2+0*1`, which is `210`. Note that this is similar to calculating the value of decimal numbers, where the rightmost digit is multiplied by 1, the digit next to that by 10, the digit next to that by 100, etcetera, and adding up all those resulting values.

When bits are numbered, by convention numbering starts at zero at the rightmost end, and numbers are increased when counting to the left, i.e., the rightmost bit has number 0, the bit next to that has number 1, the bit next to that has number 2, etcetera. The reason is that the rightmost bit represents the value `2**0` (which, in case you forgot, equals `1`), the bit next to it the value `2**1`, the bit next to that `2**2`, etcetera.  

    Example byte            1        1        0        1        0        0        1        0
    Number of bit           7        6        5        4        3        2        1        0
    Represented value    2**7     2**6     2**5     2**4     2**3     2**2     2**1     2**0
    Byte value         1*2**7 + 1*2**6 + 0*2**5 + 1*2**4 + 0*2**3 + 0*2**2 + 1*2**1 + 0*2**0 = 210

**Exercise**: Write some code that calculates the decimal number represented by a binary string of 8 ones and zeroes. The nicest solution uses a loop, a multiplier, and a total. The total starts at 0. The multiplier (which is the represented value in the example above) starts at 1, and every time the loop is traversed it is multiplied by 2. The loop processes the string from right to left (or the reversed string from left to right), and if the character encountered is a "1", it adds the multiplier to a total. This will end up with the number represented by the string as the total.

In [ ]:
# Translating a bit-string.

bitstring = "11010010"


The lowest number that can be expressed by a byte is `00000000`, which equals zero. The highest is `11111111`, which equals 255. Thus, there are 256 different values that can be expressed by one byte.

### Hexadecimal counting

Programmers often use hexadecimal numbers. The hexadecimal counting system uses digits that can have 16 different values; traditionally, these digits are `0` to `9` and `A` to `F`. I.e., when you convert a hexadecimal digit to a decimal value, `0` to `9` remain `0` to `9`, `A` is `10`, `B` is `11`, `C` is `12`, `D` is `13`, `E` is `14`, and `F` is `15`. To translate a hexadecimal number to a decimal value, you multiply the rightmost digit by `1`, the digit next to it by `16`, the digit next to that by `256` (i.e., `16*16`), etcetera, and add up the values. For instance, the hexadecimal number `C0DE` is `12*4096+0*256+13*16+14*1`, which is `49374`. The reason that programmers like using hexadecimal numbers is that one byte can be expressed by exactly two hexadecimal digits, i.e., one byte can take on the hexadecimal values from `00` to `FF`.

**Exercise**: Write some code that calculates the decimal number represented by a hexadecimal string of 4 hexadecimal digits. 

In [ ]:
# Translating a hexadecimal string

hexstring = "C0DE"


As relayed in a previous chapter, if you want to put a character represented by a hexadecimal number in a string, you can do so with the escape sequence `\xNN`, where `NN` stands for two hexadecimal digits.

In [ ]:
print( "\x48\x65\x6C\x6C\x6F\x2C\x20\x77\x6F\x72\x6C\x64\x21" )

### Character encoding

The most basic character encoding mechanism is ASCII. The ASCII table was shown in Chapter 11, including hexadecimal codes. You may have noticed that the codes used ran from (hexadecimal) `20` to `7E`. The codes below `20` are used for special sequences (such as the newline character). The code `7F` usually represent the `Del` key. No other codes are in use, which means that all ASCII characters can be represented by 7 bits, or the 8-bit sequences `00000000` to `01111111`.

While computers use bytes as basic data unit, the ASCII character set does not use 128 of all the values that can be stored in a byte. All these unused bytes have a `1` as their leftmost bit. Naturally, some character encodings were introduced that assign a character to all 256 different values that a byte can take. A typical one is `latin-1`, which is discussed in Chapter 17. Unfortunately, not all encoding mechanisms assign the same characters to the numbers between 128 and 255. However, all encoding mechanisms in use today at least have the basic ASCII characters for the values 0 to 127. 

Python is based on Unicode encoding. Specifically, it uses UTF-8 as character encoding mechanism (discussed in Chapter 11). UTF-8 encoding works as follows:

- A byte that has a zero as leftmost bit is an ASCII character.
- A byte that has a 1 as leftmost bit is the start of a sequence of multiple bytes that represent one character. The sequence consists of a leading byte (the leftmost byte) and one or more continuation bytes.
- For a multibyte sequence, the leading byte has, from left to right, several bits with value 1, followed by a bit with value zero, followed by the remaining bits. The length of the total multibyte sequence is as many bytes as there are bits with value 1 to the left of the leftmost zero. E.g., if the leading byte has value `1110xxxx` (where each `x` is some bit value), the whole sequence is three bytes long. This includes the leading byte. The minimum sequence length is two bytes, and the maximum sequence length is six bytes (the leading byte will then be `1111110x`).
- Each continuation byte has `10` as the two leftmost bits.
- In practice, UTF-8 encoding is restricted to at most 4-byte sequences, and some of the 4-byte sequences have been excluded.

This means that UTF-8 can express a great many different characters. However, it also means that, due to the way characters are encoded, some bit patterns do not express UTF-8 characters. While any bit pattern expresses a legal string with `latin-1` encoding, it is possible to construct a bit pattern that does not express a legal UTF-8 encoding. This may cause those annoying `UnicodeDecodeError`s when reading files.

### Number encoding

The way that numbers are encoded as bit patterns is somewhat tricky, and in general you do not need to bother with it. You should know that positive integers are always encoded as multi-byte patterns, that have a zero as their leftmost bit. The rest of the pattern is like you would expect, and as explained above. 

Negative numbers, however, are encoded rather differently. They use the so-called "two's complement" system. When a negative number is encoded, first the absolute value of that number (i.e., the positive version) is taken. From this number, all the bits are "flipped", i.e., every 1 becomes a zero and every zero becomes a 1. Finally, 1 is numerically added to the result. The bit pattern of a negative number therefore always has a 1 as its leftmost bit. 

For example, to encode `-1`, first the bit pattern of `1` is taken, which is `...00000001`. All the bits are flipped, which gives `...11111110`. Finally, 1 is added the the result, which gives `...11111111`. Thus, `-1` is encoded as a sequence of only `1`s. 

As for floating point numbers, these use scientific notation, whereby part of the multi-byte pattern is used as exponent.

The reason that I am explaining all of this, is to indicate that if you want to handle bit patterns in a Python program, and you want to treat these patterns as numbers, you best work only with positive integers, as the bit patterns of those are easily understood.

---

## Manipulating bits

Python offers a variety of operators that allow the manipulation of data items at the level of bits. These are the following:

    <<      shift left
    >>      shift right
    &       bitwise and
    |       bitwise or
    ~       bitwise not
    ^       bitwise exclusive or
    
They are used as follows.
    

### Shifting bits

When you have a data item, you can use the `<<` and `>>` to shift its bits to the left or right. `x<<y` shifts the bits of `x` by `y` places to the left, bringing in zeroes from the right. `x>>y` shifts the bits of `x` by `y` places to the right, copying the leftmost bit of `x` at the left while shifting, and losing the bits of `x` at the right. `x` and `y` must both be numbers.

For example, the exclamation mark `!` has decimal code `33`, which is written as `00100001` in binary. Shifting this pattern one place to the left gives `01000010`, i.e., `66` in decimal, which is the code for the capital `B`. You can reverse this by shifting the pattern of `B` one place to the right.

In [ ]:
code = "!"
print( chr(ord(code)<<1) )

code = "B"
print( chr(ord(code)>>1) )

You might have noticed that shifting a number one place to the left amounts to doubling the number, while shifting it one place to the right amounts to halving it (while rounding down). Indeed, you can double the value expressed by a bit pattern by placing a zero to the right of it -- and you can halve it (using integer division) by removing the rightmost bit.

In [ ]:
print( "345 quadrupled makes", 345<<2 )
print( "345 divided by 8 makes", 345>>3 )

### Bitwise `and`

The bitwise `and` operator (`&`) takes two bit patterns, and produces a new pattern that is all zeroes, except for those places where both bit patterns had a `1`, which will then also have a `1` in the output pattern. For instance, if the input patterns are the number 11 (`00001011`) and the number 6 (`00000110`), then the bitwise `and` operator produces the pattern `00000010`, which is the number 2.

In [ ]:
print( 11 & 6 )

**Exercise**: The bitwise `and` is an easy way to take (positive) numbers modulo a power of 2. For instance, if you want to take a number modulo 16, this is the same as performing the bitwise `and` on the number with the 15, which is `00001111`. Check that the value of 345 modulo 32 is the same as taking `345 & 31`. 

In [ ]:
# Bitwise and as modulo.


### Bitwise `or`

The bitwise `or` operator (`|`) takes two bit patterns, and produces a new pattern that is all ones, except for those places where both bit patterns had a `0`, which will then also have a `0` in the output pattern. For instance, if the input patterns are the number 11 (`00001011`) and the number 6 (`00000110`), then the bitwise `or` operator produces the pattern `00001111`, which is the number 15.

In [ ]:
print( 11 | 6 )

**Exercise**: To set a single bit in a pattern to the value 1 (this is usually called "setting a bit"), you can use the bitwise `or` and a pattern that consists of only zeroes, except for a 1 in the spot where you want to set the bit. An easy way to create a bit pattern with only one bit set, is to start with the number 1, and use the shift-left operator to shift that bit to the left as far as you need. Now take a number and set the bit with index 7 (i.e., the eight bit from the right) to `1`.

In [ ]:
# Setting a bit.
number = 11


### Bitwise `not`

The bitwise `not` operator (`~`) is placed in front of a bit pattern, and then produces a new pattern that has all the bits of the original pattern "flipped", i.e., each zero becomes a `1` and each `1` becomes a zero. For instance, if the input pattern is the number 11 (`00001011`), then the bitwise `not` produces the pattern `11110100`, which is the number -12. If you wonder why it is -12 and not -11: this is the result of the two's complement encoding, which I explained above. Don't worry too much about it.

In [ ]:
print( ~11 )

**Exercise**: To clear a single bit in a pattern (i.e., setting it to the value zero), you can use the bitwise `and` and a pattern that consists of only `1`s, except for a zero in the spot where you want to clear the bit. An easy way to create a bit pattern consisting on only ones, except for a zero in the intended spot, is to start with the number 1, and use the shift-left operator to shift that bit to the left as far as you need. Then invert the pattern with the bitwise `not` operator. Now take a number and clear the bit with index 3 (i.e., the fourth bit from the right).

In [ ]:
# Clearing a bit.
number = 11


### Bitwise `xor`

The bitwise exclusive `or`, or "`xor`", operator (`^`) takes two bit patterns, and produces a new pattern that has a zero in all places where the two bit patterns have the same bit, and a 1 in all places where the two bit patterns have different bits. For instance, if the input patterns are the number 11 (`00001011`) and the number 6 (`00000110`), then the bitwise `xor` operator produces the pattern `00001101`, which is the number 13.

In [ ]:
print( 11 ^ 6 )

**Exercise**: The bitwise `xor` operator provides an easy way to encrypt numbers. Take a bit pattern, and call it the "mask". Apply the mask to a number using the `xor`. This gives a new number, which is the encrypted number. Somebody who does not know the mask, can't tell what the original number was. However, someone who does know the mask, can easily get the original number back, by applying the mask once more. Try this.

In [ ]:
# Encoding using xor.


### Precedence of bitwise operators

Warning: the precedence of bitwise operators is *not* that they are handled before other operators. Make sure that you use parentheses to order the operators when you use bitwise operators in a calculation. For instance, you might think that `1<<1 + 2<<1` is the same as `1*2 + 2*2`, but in actuality it is evaluated as `(1<<(1+2))<<1`, or `1*8*2`.

In [ ]:
print( 1<<1 + 2<<1 )
print( (1<<1) + (2<<1) )

---

## Usefulness of bitwise operations

Anything that you can do with bitwise operators, you can also do with general calculations, with the advantage of general calculations that they can do much more than bitwise operators. So what is the use of bitwise operators?

Bitwise operations are incredibly fast. Much, much faster than regular calculations. So should you use them when making calculations, when it is opportune to do so? The answer is no, for two reasons:

- Python is already smart enough to recognize that some calculations can be executed using bitwise operators, so it will make the conversion for you.
- If you really want a fast program, you should not use Python at all.

Another use that is often mentioned, is that they facilitate storing boolean values in a small storage space. For instance, if I have eight booleans that I want to store, I can use a tuple of eight booleans, which amounts to at least eight bytes of space, or encode all eight of them in one byte using bitwise operators. However, in today's computers space is of little concern, so only if you are talking about huge, HUGE data collections you might get worried about space.

So what is the use of bitwise operators then? They are actually of fairly little use, unless you have to create programs that need to work "close to the machine". Occasionally you have to deal with data structures that are most naturally handled using bitwise operators.

To give an example: colors are usually encoded as three bytes, for the red, green, and blue channel. A color number is thus a three-byte number. Bitwise operators are a natural way to distinguish the separate color channels from a color number. Here is a function that does that:

In [ ]:
def getRGB( color ):
    blue = color & 255
    green = (color >> 8) & 255
    red = (color >> 16) & 255
    return red, green, blue

r, g, b = getRGB( 223567 )
print( "red={}, green={}, blue={}".format( r, g, b ) )

For someone who knows about color encoding, such a function reads well.

---

## What you learned

In this chapter, you learned about:

- Binary and hexadecimal counting
- Character and number encoding
- Bitwise operators `<<`, `>>`, `&`, `|`, `~`, and `^`

---

## Exercises

### Exercise 19.1

Encode a string using the bitwise exclusive `or` (`xor`) and the pattern `00101010` as mask. Display the resulting string. Then decode it, and display the decoded string, which should be the same as the original string.

In [ ]:
# XOR encoding.


### Exercise 19.2

Write a function that gets an integer, a boolean, and a number. The integer is used to store booleans. Each bit in the integer represents `True` or `False`. The bits of the integers are numbered as the convention indicates, with the rightmost bit having number zero, the bit next to that number 1, etcetera. If the boolean parameter is `True`, the function sets the bit corresponding to the number parameter in the integer to `1`. If the boolean parameter is `False`, the function clears the bit corresponding to the number parameter in the integer (i.e., it sets it to `0`). The function then returns the integer.

Also write a function that gets an integer and a number as parameters, and returns `True` if the bit corresponding to the number is set to `1`, and `False` otherwise.

To test the function, it helps to create an extra function that displays the bits in the number. The display function can make use of the function that gets the bit values.

In [ ]:
# Storing booleans in an integer.


---

End of Chapter 19. Version 1.2. 